In [4]:
from modules.utils import load_yaml, save_pickle, save_json, load_csv, save_csv
from modules.datasets import QADataset
from models.utils import get_model

from transformers import ElectraTokenizerFast, AutoTokenizer

from torch.utils.data import DataLoader

from datetime import datetime, timezone, timedelta
from tqdm import tqdm
import numpy as np
import pandas as pd
import random
import os
import torch

In [5]:
# Config
PROJECT_DIR = os.path.dirname('.')
predict_config = load_yaml(os.path.join(PROJECT_DIR, 'config', 'predict_config.yml'))

# Serial
train_serial = predict_config['TRAIN']['train_serial']
kst = timezone(timedelta(hours=9))
predict_timestamp = datetime.now(tz=kst).strftime("%Y%m%d_%H%M%S")
predict_serial = train_serial + '_' + predict_timestamp

# Predict directory
PREDICT_DIR = os.path.join(PROJECT_DIR, 'results', 'predict', predict_serial)
os.makedirs(PREDICT_DIR, exist_ok=True)

# Data Directory
DATA_DIR = predict_config['DIRECTORY']['dataset']

# Train config
RECORDER_DIR = os.path.join(PROJECT_DIR, 'results', 'train', train_serial)
train_config = load_yaml(os.path.join(RECORDER_DIR, 'train_config.yml'))

# SEED
torch.manual_seed(predict_config['PREDICT']['seed'])
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(predict_config['PREDICT']['seed'])
random.seed(predict_config['PREDICT']['seed'])

# GPU
os.environ['CUDA_VISIBLE_DEVICES'] = str(predict_config['PREDICT']['gpu'])
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [9]:
# Load tokenizer

tokenizer_dict = {'ElectraTokenizerFast': AutoTokenizer}
tokenizer = tokenizer_dict[train_config['TRAINER']['tokenizer']].from_pretrained(train_config['TRAINER']['pretrained'])

# Load data

test_dataset = QADataset(data_dir=os.path.join(DATA_DIR, 'test.json'), tokenizer = tokenizer, max_seq_len = 512, mode = 'test')

question_ids = test_dataset.question_ids

BATCH_SIZE = train_config['DATALOADER']['batch_size']

test_dataloader = DataLoader(dataset=test_dataset,
                            batch_size=BATCH_SIZE,
                            num_workers=train_config['DATALOADER']['num_workers'],
                            shuffle=False,
                            pin_memory=train_config['DATALOADER']['pin_memory'],
                            drop_last=train_config['DATALOADER']['drop_last'])

# Load model
model_name = train_config['TRAINER']['model']
# model_args = train_config['MODEL'][model_name]
model = get_model(model_name=model_name, pretrained=train_config['TRAINER']['pretrained']).to(device)

checkpoint = torch.load(os.path.join(RECORDER_DIR, 'model.pt'))

if train_config['TRAINER']['amp'] == True:
    from apex import amp
    model = amp.initialize(model, opt_level='O1')
    model.load_state_dict(checkpoint['model'])
    amp.load_state_dict(checkpoint['amp'])

else:
    model.load_state_dict(checkpoint['model'])

model.eval()
pred_df = load_csv(os.path.join(DATA_DIR, 'sample_submission.csv'))

for batch_index, batch in enumerate(tqdm(test_dataloader, leave=True)):
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)

    # Inference
    outputs = model(input_ids, attention_mask=attention_mask)

    start_score = outputs.start_logits
    end_score = outputs.end_logits

    start_idx = torch.argmax(start_score, dim=1).cpu().tolist()
    end_idx = torch.argmax(end_score, dim=1).cpu().tolist()

    y_pred = []
    for i in range(len(input_ids)):
        if start_idx[i] > end_idx[i]:
            output = ''

        ans_txt = tokenizer.decode(input_ids[i][start_idx[i]:end_idx[i]+1]).replace('#','')

        if ans_txt == '[CLS]':
            ans_txt == ''

        y_pred.append(ans_txt)

    q_end_idx = BATCH_SIZE*batch_index + len(y_pred)
    for q_id, pred in zip(question_ids[BATCH_SIZE*batch_index:q_end_idx], y_pred):
        pred_df.loc[pred_df['question_id'] == q_id,'answer_text'] = pred

save_csv(os.path.join(PREDICT_DIR, 'prediction.csv'), pred_df)

Some weights of the model checkpoint at monologg/koelectra-base-v3-discriminator were not used when initializing ElectraForQuestionAnswering: ['discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.bias']
- This IS expected if you are initializing ElectraForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForQuestionAnswering were not initialized from the model checkpoint at monologg/koelectra-base-v3-discriminator and are newly initialized: ['qa_outputs.bias', 

In [10]:
pred_df

,question_id,answer_text
0,QUES_cyOI2451l1,한국원자력안전기술원
1,QUES_pz2vbWpWWo,가출청소년 문제
2,QUES_1g3jI4y7eo,고등학생
3,QUES_qzwOZwaeeY,Prime Air
4,QUES_hfdtXCtdzf,장애인케어서비스
...,...,...
1621,QUES_JtsKBSQITG,고령층 임금근로자
1622,QUES_IajaDLmxvq,가처분소득
1623,QUES_lR6hjzsptY,대체보육서비스 비용
1624,QUES_ACwZJGYBfp,IBM Food Trust Labeyrie사


In [11]:
test_df = pd.read_csv('test_raw.csv')

In [28]:
total_count = 0
correct_count = 0
for i, row in test_df.iterrows():
    pred = pred_df[pred_df['question_id']==row.question_id]['answer_text'].values[0]
    row.pred = pred
    if row.pred.strip(' ') == row.answer.strip(' '):
        correct_count += 1
    else:
        print(row.answer, "!=", row.pred)
    total_count += 1
print(total_count, correct_count)
print('accuracy =', correct_count/float(total_count))

탐색기 != 고등학생
사례 담당자 != 사례 담당자의 배치가 필요하다. 이를 위해서는 우선적으로 현재 활동지원서비스 중개기관과는 별도로 활동지원서비스를 모니터링 할 수 있는 새로운 기관이나 인력의 배치가 필요하다. 대표적인 방법이 국민연금공단에 장애인 사례담당인력
정시도착률 != 리포팅 항공사 ’ 들의 운항 정보는 항공교통이용자 보고서에 기존처럼 계속 포함되고, 이에 더하여 ‘ 마케팅 항공사 ( marketing carrier ) ’ 들의 항공 서비스 품질 데이터가 추가된 것이다. 예로, 지난 7월 한 달 동안의 데이터를 바탕으로 마련된 최신 항공교통이용자 보고서는 정시도착률이 가장 높았던 ‘ 마케팅 항공사 ’ 를 순서대로 하와이안항공 네트워크
유나이티드익스프레스 != 간선 항공사
ADS-B 리베이트 프로그램 != ADS - B 리베이트 프로그램
인센티브 경매 != 자발적 권리 반납은 주파수에 대한 완전한 권리의 포기뿐만 아니라 타 대역으로의 주파수 이동, 주파수 공유를 포함하고 있다. 따라서 기존의 주파수 할당에서 볼 수 없는 주파수 재배치 과정
3년 이하 != 3년
물리적 세계 및 프로세스 != 디지털 트윈
대외적 구속력 != 작용법적 근거 마련
잠열 저장 != 현열 저장
스타트업 != 관광스타트업
2014년 != 
선택적/수직적 수단 != 선택적 / 수직적 수단
시민권 행사 != 
유교윤리 != 간음행위
위법성조각사유의 법적 한계 != 
위-수탁의 관계 != 
근린재생형 != 도시재생 활성화 미 지원에 관한 특별법
주요 수종별 및 임령별 != IPCC GPG 산출방법의 검토 및 적용성 검증이 필요하다. 산림 내 잔존하는 바이오매스 부후에 대한 보다 과학적인 접근법이 요구되며, 고사목에 대한 탄소저장량 추정을 위한 모델 구축이 필요하다. 토양탄소 저장량 / 변화량 도출을 위한 모니터링 시스템 구축 및 토양탄소모델 개발
범죄피해재산 != 범죄피해재산의 피해자 환부제도
진실과 정의의 원칙 != 범죄피해재산의 회복에 관여하는 것이 법적인 관점에서 합리성이 인정된

저어새 != 강화갯벌이
근본적인 기본권 침해 요소 != 법관에 의한 통제 강화
HRD-Index != 
KIPnet 컨퍼런스 2014 != KIPnet 컨퍼런스
중국 != 제13차 5개년 계획 ” 및 “ 중국 제조 2025 전략 ” 등에 따라 이러한 구조고도화는 더 빨라지게 되어 우리 주력산업을 위협하고 있다. 이미 세계 수출시장 점유율과 같은 양적인 측면에서는 중국이 모든 산업에서 우리를 크게 넘어섰을 뿐만 아니라 동일 산업 내 품질, 기술, 신산업 대응 등과 같은 질적인 측면에서도 우리에 근접하거나 우리를 넘어선 부분도 있다. 우리 산업이 중국과 경쟁하고 중국시장에서 진출을 확대하기 위해서는 과거와는 다른 전략이 필요할 것이다. 생산기지로서의 역할보다 시장 공략을 위한 중국 내 R & D 기능 등이 강조되는 새로운 가치사슬 분업전략
텃제 != 개기제
대곡 문화 != 곡비가 전통사회의 애환으로 각인된 것은 최근의 일이다. 『 국어사전 』 에는 ‘ 양반의 장례 때 주인을 대신하여 곡하던 계집종 ’ 이라고 풀고 사례로 『 혼불 』 ( 1981 ~ 1996 ) 이라는 소설에서 ‘ 곡비의 울음소리 온 밤 내내 구슬픈 물굽이를 … ’ 이라는 대목을 들고 있다. 또한 2014년 3월 방영된 ( KBS2 ) ‘ 곡비 ’ 와 2015년 4월 ‘ 총체적 난극 - 곡비 ’ 라는 연극이 그 주인공이다. 그러나 이들은 하나같이 ‘ 곡비 = 천민 ’ 이라 는 등식으로만 바라보고 있어 곡비와 대곡문화의 실체와 그 본래 의미를 읽어내지 못했다. 실제 곡비는 극히 드문 일이었고, 소설 외에 어떤 형태로 존재했는지 알 수 없기에 일반화할 수는 없다. 우리에게 곡비라는 슬픈 역사로 알려져 버린 ‘ 대곡 ’ 은 슬픔에 빠진 상주를 보호하고, 고인에 대한 여운을 풀어주며, 자식 된 도리를 일깨우는 배려의 풍습이었다. 그래서인지 대곡 문화
심리학자 프랭크 미너스Frank minirth 박사 != 
정보기관과 수사기관 != 테러범죄 예방을 실패했다는 개념이다. 이러한 반성적 고려를 통해 미국은 ‘

,question_id,context,question,answer,pred,Unnamed: 5
0,QUES_cyOI2451l1,생활방사선법 제23조(생활주변방사선 안전관리 실태조사 및 분석) 제1항에 따른 안전...,어디서 모나자이트 취급사업장을 생활주변방사선 실태조사 대상으로 뽑았어,한국원자력안전기술원,한국원자력안전기술원,NaN
1,QUES_pz2vbWpWWo,또한 최근 위기청소년의 증가와 더불어 위기청소년 사회안전망 구축사업에서 가출청소년 ...,어떤 문제가 위기청소년 사회안전망 형성사업에서 중요한 부분인가,가출청소년 문제,가출청소년 문제,NaN
2,QUES_1g3jI4y7eo,"고등학교 시기는 삶의 경험 속에서 지속적인 자아 탐색, 직업정보 검색 및 평가 등의...",어떤 시기 때문에 고등학생 시기가 혼란스럽고 정서적 불안을 느낄 가능성이 커,탐색기,고등학생,NaN
3,QUES_qzwOZwaeeY,흥미롭게도 물류로봇과 배송로봇은 모두 세계최대의 전자상거래 기업인 Amazon에 의...,Amazon은 2013년 무슨 드론을 이용한 배송 서비스를 발표했지,Prime Air,Prime Air,NaN
4,QUES_hfdtXCtdzf,미국의 사례를 통해 살펴보면 현행 우리나라의 활동지원서비스는 장애인케어서비스와 유사...,한국의 활동지원서비스는 미국의 예를 통해 살펴보면 뭐와 비슷할까,장애인케어서비스,장애인케어서비스,NaN
...,...,...,...,...,...,...
1621,QUES_JtsKBSQITG,고령층의 자영업 비중이 감소하면서 고령층 임금근로자 비중은 증가하고 있다. 2008...,2016년 약 27프로로 꾸준히 증가하는 추세를 보인건 고령층의 무엇이지,상용직,고령층 임금근로자,NaN
1622,QUES_IajaDLmxvq,통계청의 가계동향조사에 의하면 60세 이상 노인 단독 가구의 처분가능소득(2015년...,어떤 소득이 60세 이상 독거 노인 가구의 가처분소득에서 가장 높은 비율을 차지해,공적이전소득,가처분소득,NaN
1623,QUES_lR6hjzsptY,여성의 시간배분 결정에 있어서 중요한 가격요인 중 하나가 대체보육서비스 비용이다. ...,무엇이 여성의 시간배분 결정에 있어서 주요한 가격소이 가운데 하나일까,대체보육서비스 비용,대체보육서비스 비용,NaN
1624,QUES_ACwZJGYBfp,노르웨이 Cermaq사는 IBM Food Trust Labeyrie사와 협력하여 연...,노르웨이 Cermaq사가 어느 회사와 협력하여 연어의 생산과 유통 정보시스템을 구축했니,IBM Food Trust Labeyrie사,IBM Food Trust Labeyrie사,NaN
